In [24]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.volume import ForceIndexIndicator
from ta.volume import EaseOfMovementIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.volume import NegativeVolumeIndexIndicator
from ta.trend import SMAIndicator
from ta.volatility import BollingerBands
from ta.volatility import KeltnerChannel
from ta.volatility import AverageTrueRange
from ta.volatility import DonchianChannel
from ta.others import DailyLogReturnIndicator
payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
#print(first_table.head())
sip=np.array(payload.iloc[:,0])
sip=sip[sip!='BRK.B']
sip=sip[sip!='BF.B']
df=pd.DataFrame()
dfout=pd.DataFrame()
start_date="2000-10-01"
end_date="2021-10-01"
txt = "{:%}"
wind=14
for i in range(0,100):

    stk=sip[i]
    #df = yf.download(stk, start_date, end_date,interval=intr).dropna()

    dfd = yf.download(stk, start_date, end_date,interval='1d').dropna()
    dfd['Open1']=dfd.Open.shift(1)
    dfd['Close1']=dfd.Close.shift(1)
    dfd['High1']=dfd.High.shift(1)
    dfd['Low1']=dfd.Low.shift(1)
    dfd['Volume1']=dfd.Volume.shift(1)
    dfd['Change']=dfd.Close-dfd.Close1


    dfd['fi']=ForceIndexIndicator(close=dfd.Close1,volume=dfd.Volume1).force_index()
    #dfd['efi_rsid']=RSIIndicator(close=dfd.efid,window=wind,fillna=True).rsi()
    #dfd["kd"]=StochRSIIndicator(close=dfd.Close_1,window=wind,smooth1=3, smooth2=3).stochrsi_k() 
    dfd['bbh']=BollingerBands(close=dfd.Close1).bollinger_hband()
    dfd['kch']=KeltnerChannel(close=dfd.Close1,high=dfd.High1,low=dfd.Low1).keltner_channel_hband() 
    dfd['dch']=DonchianChannel(close=dfd.Close1,high=dfd.High1,low=dfd.Low1).donchian_channel_hband() 
    dfd['bbli']=BollingerBands(close=dfd.Close1).bollinger_lband_indicator()
    dfd['kcli']=KeltnerChannel(close=dfd.Close1,high=dfd.High1,low=dfd.Low1).keltner_channel_lband_indicator() 
    dfd['atr']=AverageTrueRange(high=dfd.High1,low=dfd.Low1,close=dfd.Close1).average_true_range()
    dfd['nvi']=NegativeVolumeIndexIndicator(close=dfd.Close1,volume=dfd.Volume1).negative_volume_index()
    dfd['eom']=EaseOfMovementIndicator(high=dfd.High1,low=dfd.Low1,volume=dfd.Volume1).ease_of_movement()
    dfd['vpt']=VolumePriceTrendIndicator(close=dfd.Close1,volume=dfd.Volume1).volume_price_trend()
    dfd['dlr']=DailyLogReturnIndicator(close=dfd.Close1).daily_log_return()
    #& (dfd.volatility_dch>np.percentile(dfd.volatility_dch,60))
    #& (dfd.volatility_kch>np.percentile(dfd.volatility_kch,60))    


    #dfd['sma200']=SMAIndicator(close=dfd.Close_1,window=200,fillna=True).sma_indicator()
    #dfd['sma50']=SMAIndicator(close=dfd.Close_1,window=50,fillna=True).sma_indicator()
    #dfd['sma20']=SMAIndicator(close=dfd.Close_1,window=20,fillna=True).sma_indicator()
    #dfd['sma10']=SMAIndicator(close=dfd.Close_1,window=10,fillna=True).sma_indicator()
    #dfd["Chng"]=(dfd.Close-dfd.Open)
    #dfd['ChngOv']=dfd.Close-dfd.Close_1
    #dfd['Chng_1']=dfd.Chng.shift(1)
    #dfd['Chng_smaf']=SMAIndicator(close=dfd.Chng_1,window=5,fillna=True).sma_indicator()
    #dfd['Chng_smas']=SMAIndicator(close=dfd.Chng_1,window=20,fillna=True).sma_indicator()
    #dfd['Vol_smaf']=SMAIndicator(close=dfd.Volume_1,window=5,fillna=True).sma_indicator()
    #dfd['Vol_smas']=SMAIndicator(close=dfd.Volume_1,window=20,fillna=True).sma_indicator()
    #print(dfd.tail)
    dfd=dfd.dropna()
    x=15
    x2=5

    conditions = [ 
    (dfd['Open']>0)
    #& (dfd.bbli==1)
    #& (dfd.kcli==1)

    #& (dfd.bbh>np.percentile(dfd.bbh,70))
    #& (dfd.kch>np.percentile(dfd.kch,70))
    #& (dfd.dch>np.percentile(dfd.dch,70))
    #& (dfd.atr>np.percentile(dfd.atr,70))
    #& (dfd.nvi>np.percentile(dfd.nvi,70))   
    & (dfd.fi<np.percentile(dfd.fi,x))
    & (dfd.eom<np.percentile(dfd.eom,x))
    & (dfd.vpt<np.percentile(dfd.vpt,x))
    & (dfd.dlr<np.percentile(dfd.dlr,x))
    #& (dfd.dch>np.percentile(dfd.dch,60))
    #& (dfd.kch>np.percentile(dfd.kch,60))

    ,
    (dfd['Open']>0)
    #& (dfd.fi>np.percentile(dfd.fi,100-x))
    & (dfd.eom>np.percentile(dfd.eom,100-x2))
    #& (dfd.vpt<np.percentile(dfd.vpt,100-x))
    & (dfd.dlr>np.percentile(dfd.dlr,100-x2))

    ]

    choices = [
    1,-1
    ]
    dfd['trade_pass']=np.select(conditions,choices,default=0)
    dfd_long=dfd[dfd['trade_pass']==1]
    dfd_short=dfd[dfd['trade_pass']==-1]
    #print(dfd_long)
    fir=dfd.Open[0]
    las=dfd.Close[len(dfd.index)-1]
    long_prof=(np.sum(dfd_long.Change)+fir)/fir-1
    short_prof=(-np.sum(dfd_short.Change)+fir)/fir-1
    trade_rate_long=len(dfd_long.index)/(len(dfd.index))
    trade_rate_short=len(dfd_short.index)/(len(dfd.index))
    hold_wr=len(dfd[dfd.Change>0])/(len(dfd.index))
    long_wr=len(dfd_long[dfd_long.Change>0])/(len(dfd_long.index))
    short_wr=len(dfd_short[dfd_short.Change<0])/(len(dfd_short.index))
    hold_prof=(las)/(fir)-1
    #print(hold)
    #print(long_prof)
    #print(trade_rate)
    #print(hold_wr)
    #print(trade_wr)
    #print(max(dfd.kd))
    temp=pd.Series([stk,hold_prof,long_prof,short_prof,hold_wr,long_wr,short_wr,trade_rate_long,trade_rate_short])
    dfout=dfout.append(temp,ignore_index=True)

print(dfout)
fin=[]
for j in range (1,len(dfout.columns)):
    temp=(np.average(dfout.iloc[:,j]))
    fin.append(temp)
print(fin)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********